# Model 1 : XGBoost


 This notebook performs classification of Reddit posts into popularity buckets using XGBoost, leveraging TF-IDF features extracted from post titles along with other structured categorical features. The workflow includes data preprocessing, baseline evaluation, model training,cross-validation, hyperparameter tuning with Optuna, and model evaluation.

In [39]:
# Import all the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import shap
from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix 
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sklearn.metrics import (
    cohen_kappa_score,
    matthews_corrcoef,
    log_loss 
)


### Data Preprocessing

In [40]:
# Load dataset

df = pd.read_csv("../data/cleaned_reddit_posts.csv")

df.head()


,subreddit,id,title,selftext,score,num_comments,flair,upvote_ratio,is_self,nsfw,author,sort_type,popularity_bucket,created_hour,media_type
0,technology,1lvds7w,students can’t use ai to cheat on standardized...,NaN,2,2,Artificial Intelligence,1.00,False,False,ubcstaffer123,new,low,8,external_link
1,technology,1lvdi5e,instagram wrongly accuses some users of breach...,NaN,9,2,Social Media,1.00,False,False,zsreport,new,low,8,external_link
2,technology,1lvcxoa,turkey blocks x's grok chatbot for alleged ins...,NaN,18,3,Social Media,0.91,False,False,BreakfastTop6899,new,low,7,external_link
3,technology,1lvai0d,globalfoundries to make risc-v cpus — fab acqu...,NaN,18,1,Hardware,0.83,False,False,jhansonxi,new,low,5,external_link
4,technology,1lv9syt,rubio impersonation campaign underscores broad...,NaN,25,3,Artificial Intelligence,0.82,False,False,BreakfastTop6899,new,low,4,external_link


We first drop unnecessary columns, such as id and score (we drop score as it has been converted to popularity_bucket field). 
We also drop the columns upvote_ratio and num_comments, as these fields are highly correlated to the output and could potentially leak the output label. 
Author field is dropped as it has very high cardinality (5032 unique values out of 10047 rows), and would not aid in prediction.

In [41]:
# Drop columns that are not used for prediction

drop_cols = ['score', 'upvote_ratio', 'sort_type', 'id', 'author', 'selftext','num_comments']
df = df.drop(columns=drop_cols)

In [42]:
# Apply TF-IDF vectorization on the 'title' column

tfidf = TfidfVectorizer(max_features=1000)
X_title_tfidf = tfidf.fit_transform(df['title']).toarray()

In [43]:
# Encode categorical features using Label Encoding

label_enc_cols = ['subreddit', 'flair', 'media_type']
for col in label_enc_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

In [44]:
# Encode the target variable 'popularity_bucket'

target_le = LabelEncoder()
df['popularity_bucket'] = target_le.fit_transform(df['popularity_bucket'])

In [45]:
# Convert TF-IDF output to DataFrame for easy concatenation

tfidf_df = pd.DataFrame(
    X_title_tfidf,
    columns=[f'tfidf_{i}' for i in range(X_title_tfidf.shape[1])],
    index=df.index
)

In [46]:
# Prepare structured feature DataFrame by removing target and raw text

structured_df = df.drop(columns=['popularity_bucket', 'title'])


In [47]:
# Split into features and labels
# Concatenate TF-IDF features with structured features to form final feature matrix X

X = pd.concat([tfidf_df, structured_df], axis=1)
y = df['popularity_bucket']

### Split the data

In [48]:
# Split data into training + testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,                # input and output
    test_size=0.2,       # 20% for testing, 80% for training
    random_state=42,
    stratify=y           # ensures same label distribution in train & test
)

### Baseline evaluation using majority class prediction

 We create a simple baseline model that always predicts the most common class from the training data. This helps us understand how well a real model should perform by comparison.

In [49]:
# Baseline: Naive majority class 


print(df["popularity_bucket"].value_counts())

# Identify majority class in training data
majority_class = Counter(y_train).most_common(1)[0][0]
print("Majority class in training data:", majority_class)

# Predict majority class for all test samples as a naive baseline
y_pred_baseline = np.full_like(y_test, fill_value=majority_class)

# Evaluate performance
print("\nBaseline Performance (Majority Class):")
print("Accuracy:", round(accuracy_score(y_test, y_pred_baseline), 4))
print("F1 Score (macro):", round(f1_score(y_test, y_pred_baseline, average='macro'), 4))
print("Cohen's Kappa:", round(cohen_kappa_score(y_test, y_pred_baseline), 4))
print("Matthews Correlation Coefficient (MCC):", round(matthews_corrcoef(y_test, y_pred_baseline), 4))


popularity_bucket
0    3415
1    3316
2    3316
Name: count, dtype: int64
Majority class in training data: 0

Baseline Performance (Majority Class):
Accuracy: 0.3398
F1 Score (macro): 0.1691
Cohen's Kappa: 0.0
Matthews Correlation Coefficient (MCC): 0.0


### Train Initial XGBoost Model

We train a basic XGBoost classifier on the training data and evaluate its performance on the test set. This gives us a strong starting point before doing any tuning or cross-validation.

Initialize the Model
- We create an instance of XGBClassifier with the following key settings:

    - `use_label_encoder`=`False`: Avoids warnings in new XGBoost versions.

    - `eval_metric`=`'mlogloss'`: Specifies the evaluation metric suitable for multiclass classification.

    - `tree_method`=`'hist'` and `device`=`'cuda'`: Enable fast training using GPU and histogram-based tree building.

    - `random_state`=`42`: Ensures reproducibility.

Train the Model

- The model is trained on the full training set `X_train` and `y_train`.

Predict on the Test Set
- We use `predict()` on `X_test` to get predictions for unseen data.

Evaluate the Performance

In [50]:
# Train an XGBOOST Model

# Initialize the model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', tree_method="hist", device="cuda", random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)



c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [51]:
# Evaluate on the test set

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate
print("Baseline XGBoost Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score (macro):", f1_score(y_test, y_pred, average='macro'))
print("Cohen's Kappa:", cohen_kappa_score(y_test, y_pred))
print("Matthews Correlation Coefficient (MCC):", matthews_corrcoef(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Baseline XGBoost Performance:
Accuracy: 0.6587064676616915
F1 Score (macro): 0.6589554675730152
Cohen's Kappa: 0.4878083027311988
Matthews Correlation Coefficient (MCC): 0.4889894433662448

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.72      0.69       683
           1       0.74      0.64      0.68       664
           2       0.59      0.62      0.60       663

    accuracy                           0.66      2010
   macro avg       0.66      0.66      0.66      2010
weighted avg       0.66      0.66      0.66      2010



###  Simple Validation Split

 To check how well our XGBoost model performs before full training, we do a basic train-validation split.
- First, we split the original training data again: 80% for training and 20% for validation. This helps us evaluate the model on unseen data during the training phase itself (not the test set).
- We train a new XGBoost model on this smaller training set (`X_train_final`, `y_train_final`).
- Then we make predictions on the validation set (`X_val`) and calculate performance metrics:
    - Accuracy – how many predictions were correct
    - F1 Score (macro) – balances precision and recall across all classes
    - Cohen's Kappa – measures agreement between predicted and true labels
    - Matthews Correlation Coefficient (MCC) – a balanced metric even for imbalanced data


In [52]:
# Simple CV

# Step 1: Split into train (80%) and test (20%) - already done

# Now: Split train into train (80%) and val (20%)
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train  
)

# Step 2: Train on training set
xgb_model_simple_cv = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss')
xgb_model_simple_cv.fit(X_train_final, y_train_final)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [53]:
# Step 3: Evaluate on validation set
y_val_pred = xgb_model_simple_cv.predict(X_val)


print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation F1 Score (macro):", f1_score(y_val, y_val_pred, average='macro'))
print("Validation Cohen's Kappa:", cohen_kappa_score(y_val, y_val_pred))
print("Validation Matthews Correlation Coefficient (MCC):", matthews_corrcoef(y_val, y_val_pred))

Validation Accuracy: 0.650497512437811
Validation F1 Score (macro): 0.6521165917867692
Validation Cohen's Kappa: 0.47564923290804473
Validation Matthews Correlation Coefficient (MCC): 0.47647637337113347


### 5-Fold Cross Validation

To get a better estimate of how well our XGBoost model will perform on new, unseen data, we use 5-fold cross-validation.

How it works?

- We divide the original training set (`X_train`, `y_train`) into 5 equal parts (folds) using `StratifiedKFold`.

- For each fold (fold 1 to fold 5):

    - Four parts are used for training → `X_tr`, `y_tr`

    - The remaining one part is used for validation → `X_val_fold`, `y_val_fold`

- This process ensures that:

    - Every sample is used once for validation. The class distribution remains balanced in all folds.

- Inside each fold:

    - The model is trained on `X_tr`, `y_tr`.

    - Predictions are made on `X_val_fold`.

    - We compute the following metrics:

        - `acc`: Accuracy

        - `f1`: F1 Score (macro)

        - `kappa`: Cohen's Kappa

        - `mcc`: Matthews Correlation Coefficient
    
- Final step:

    - After all 5 folds, we calculate the average and standard deviation of these metrics:

        - np.mean(accuracies) ± np.std(accuracies)

        - np.mean(f1_scores) ± np.std(f1_scores) and so on for Kappa and MCC

This gives us a robust estimate of model performance across different subsets of the data.

In [54]:
# 5-Fold CV

# Set up Stratified K-Fold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store metrics for each fold
accuracies = []
f1_scores = []
kappas = []
mccs = []

# Loop through each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), 1):

    # Split data into train and validation based on fold
    X_tr, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Initialize model
    model = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', use_label_encoder=False, tree_method="hist", device="cuda", random_state=42)

    # Train on training fold
    model.fit(X_tr, y_tr)

    # Predict on validation fold
    y_pred_fold = model.predict(X_val_fold)

    # Evaluate metrics
    acc = accuracy_score(y_val_fold, y_pred_fold)
    f1 = f1_score(y_val_fold, y_pred_fold, average='macro')
    kappa = cohen_kappa_score(y_val_fold, y_pred_fold)
    mcc = matthews_corrcoef(y_val_fold, y_pred_fold)

    accuracies.append(acc)
    f1_scores.append(f1)
    kappas.append(kappa)
    mccs.append(mcc)

    print(f"Fold {fold} - Acc: {acc:.4f} | F1: {f1:.4f} | Kappa: {kappa:.4f} | MCC: {mcc:.4f}")

# After all folds
print("\nK-Fold CV Results (5 folds):")
print(f"Avg Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Avg F1 Score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"Avg Cohen's Kappa:{np.mean(kappas):.4f} ± {np.std(kappas):.4f}")
print(f"Avg MCC:          {np.mean(mccs):.4f} ± {np.std(mccs):.4f}")



c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 1 - Acc: 0.6604 | F1: 0.6612 | Kappa: 0.4905 | MCC: 0.4912


c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 2 - Acc: 0.6549 | F1: 0.6562 | Kappa: 0.4822 | MCC: 0.4833


c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:27:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 3 - Acc: 0.6397 | F1: 0.6397 | Kappa: 0.4592 | MCC: 0.4608


c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:28:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:28:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:28:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 4 - Acc: 0.6534 | F1: 0.6525 | Kappa: 0.4797 | MCC: 0.4809


c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:28:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:28:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:28:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 5 - Acc: 0.6403 | F1: 0.6427 | Kappa: 0.4604 | MCC: 0.4635

K-Fold CV Results (5 folds):
Avg Accuracy: 0.6497 ± 0.0083
Avg F1 Score: 0.6505 ± 0.0081
Avg Cohen's Kappa:0.4744 ± 0.0124
Avg MCC:          0.4760 ± 0.0118


### Hyperparameter Tuning with Optuna

We use Optuna, a powerful automatic hyperparameter optimization framework, to find the best combination of hyperparameters for our XGBoost model.

Objective Function:

- We define an `objective()` function which:

    - Suggests hyperparameters for each trial (like `n_estimators`, `max_depth`, `learning_rate`, etc.)

    - Trains an XGBoost model using `X_train`, `y_train`

    - Predicts on the test set `X_test`

    - Returns the macro F1 score, which Optuna tries to maximize.

- Main hyperparameters being tuned:

    - `n_estimators`: Number of boosting rounds (trees)

    - `max_depth`: Maximum depth of each tree

    - `learning_rate`: Controls how quickly the model learns

    - `subsample` & `colsample_bytree`: Control how much of the data/features are used per tree

    - `gamma`, `reg_alpha`, `reg_lambda`: Regularization terms to reduce overfitting



- We create an Optuna study that:

    - Tries multiple hyperparameter combinations (here, `n_trials`=30)

    - Uses TPE (Tree-structured Parzen Estimator) for sampling

    - Uses Median Pruner to stop underperforming trials early


- After running 30 trials:

    - The best F1 score and corresponding parameters are printed. These can be used to train the final model.

In [ ]:
# Hyperparameter tuning using optuna

# Objective function that Optuna will optimize
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0),
        'random_state': 42,
        'eval_metric': 'mlogloss',
        'tree_method':'hist', 
        'device':'cuda'
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    f1 = f1_score(y_test, preds, average='macro')
    return f1                                           # Optuna will try to maximize this

# Create the study and run it
study = optuna.create_study(direction='maximize', sampler=TPESampler(), pruner=MedianPruner())
study.optimize(objective, n_trials=40)

# Print best hyperparameters
print("Best trial:")
print(f"  F1 Score: {study.best_value}")
print("  Best hyperparameters:")
for key, val in study.best_params.items():
    print(f"    {key}: {val}")

[I 2025-07-16 11:34:19,332] A new study created in memory with name: no-name-7ed7dade-a325-44a6-b457-70c74519d20e


c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:34:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:34:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-07-16 11:34:46,963] Trial 0 finished with value: 0.6278435121902273 and parameters: {'n_estimators': 409, 'max_depth': 3, 'learning_rate': 0.27487516950247404, 'subsample': 0.8301942550488062, 'colsample_bytree': 0.6378727933854071, 'gamma': 4.892959493619313, 'reg_lambda': 0.7064235425679269, 'reg_alpha': 9.404745993286204}. Best is trial 0 with value: 0.6278435121902273.
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:34:47] 

Best trial:
  F1 Score: 0.6616560310827827
  Best hyperparameters:
    n_estimators: 283
    max_depth: 8
    learning_rate: 0.2110999223359582
    subsample: 0.7071505395696376
    colsample_bytree: 0.7859617505621581
    gamma: 4.251667751218394
    reg_lambda: 0.9370836298331021
    reg_alpha: 0.47084711608536645


### Retrain Model with Best Hyperparameters

We use the best hyperparameters found by Optuna to train our final XGBoost model and evaluate its performance.

- Update and Prepare Best Parameters:
    - We get the best parameter set from `study.best_trial.params`.

    - Then, we add required parameters to this set:

        - `objective`: Defines the task — here, multi:softprob for multiclass classification.

        - `num_class`: Tells the model how many classes (popularity buckets) we have.

        - `use_label_encoder`: Disabled to avoid warnings.

        - `eval_metric`: We use mlogloss, a common metric for multiclass problems.

        - `tree_method` and `device`: Speed up training using histogram-based trees and GPU.

- Train Final Model

We train the XGBoost model (`final_model`) using all of the training data (`X_train`, `y_train`) and the best settings from tuning.

`predict()` gives the final class predictions on the test set.

`predict_proba()` gives the predicted probabilities for each class — needed for computing log loss.

- We then evaluate the model on:

    - Accuracy: Overall correct predictions

    - F1 Score (macro): F1 score for each class, averaged equally

    - Cohen’s Kappa: Measures agreement between predictions and true labels

    - Matthews Correlation Coefficient (MCC): A robust metric even for imbalanced classes

    - Log Loss: Penalizes incorrect predictions with high confidence

In [59]:
# Retrain the model using the best parameters
best_params = study.best_trial.params
best_params.update({
    "objective": "multi:softprob",
    "num_class": len(target_le.classes_),
    "use_label_encoder": False,
    "eval_metric": "mlogloss",
    "tree_method": "hist",
    "device": "cuda"
})

final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

y_test_pred_best = final_model.predict(X_test)
y_proba = final_model.predict_proba(X_test)

# Final test performance
print("Final Evaluation After Tuning:")
print("Accuracy:", accuracy_score(y_test, y_test_pred_best))
print("F1 Score (macro):", f1_score(y_test, y_test_pred_best, average='macro'))
print("Cohen's Kappa:", cohen_kappa_score(y_test, y_test_pred_best))
print("Matthews Correlation Coefficient (MCC):", matthews_corrcoef(y_test, y_test_pred_best))
print("Log Loss:", log_loss(y_test, y_proba))
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred_best))


c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:45:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:45:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\dsouz\DSW-Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:45:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Final Evaluation After Tuning:
Accuracy: 0.6686567164179105
F1 Score (macro): 0.6688951832706106
Cohen's Kappa: 0.5027306611907332
Matthews Correlation Coefficient (MCC): 0.504629853530596
Log Loss: 0.7875904708191223

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.73      0.70       683
           1       0.76      0.62      0.68       664
           2       0.60      0.65      0.62       663

    accuracy                           0.67      2010
   macro avg       0.68      0.67      0.67      2010
weighted avg       0.68      0.67      0.67      2010



In [60]:
# Confusion matrix plot
cm = confusion_matrix(y_test, y_test_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_le.classes_, yticklabels=target_le.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.close()

In [61]:
# Feature importance plot
importances = final_model.feature_importances_
top_idx = importances.argsort()[-20:][::-1]   # Top 20 important features
top_features = X.columns[top_idx]

sns.barplot(x=importances[top_idx], y=top_features)
plt.title("Top 20 Feature Importances")
plt.tight_layout()
plt.savefig("feature_importance.png")
plt.close()

In [62]:
# SHAP Explainer and Plots

# Create SHAP explainer
explainer = shap.Explainer(final_model)

# Compute SHAP values on test set
shap_values = explainer(X_test)

# Summary plot: global feature importance
shap.summary_plot(shap_values, X_test, feature_names=X.columns, show=False)
plt.tight_layout()
plt.savefig("shap_summary_plot.png")
plt.close()

# Dependence plot for the top feature from your feature importance
top_feature = top_features[0]
class_idx = 0  # Choose the class index you want to visualize (0, 1, or 2 for your 3 classes)
shap.dependence_plot(top_feature, shap_values.values[:, :, class_idx], X_test, feature_names=X.columns, show=False)
plt.tight_layout()
plt.savefig(f"shap_dependence_{top_feature}.png")
plt.close()


final_model.save_model("xgb_model.json")